# 🚦 YOLO Detection Server V24
### Changes:
- ✅ Removed crossing detection
- ✅ Count unique track_ids by vehicle type
- ✅ Send track counts to backend
- ✅ VN plate format validation + voting
- ✅ Red light + Lane violation

In [ ]:
# Cell 1: CONFIG
BACKEND_URL = 'https://written-taxation-traveling-edwards.trycloudflare.com'
NMS_URL = 'https://legendary-figured-liquid-june.trycloudflare.com'

BACKEND_HTTP_URL = BACKEND_URL
BACKEND_WS_URL = BACKEND_URL.replace('https://', 'wss://').replace('http://', 'ws://') + '/ws/kaggle'

DEFAULT_TRACK_LINE_Y = 50
CONFIDENCE = 0.5
FRAME_QUEUE_SIZE = 3
RESULT_QUEUE_SIZE = 10
LOG_INTERVAL = 3
VIOLATION_COOLDOWN = 15

# COCO class IDs to vehicle class names
COCO_CLASS_MAP = {
    2: 'car',
    3: 'motorcycle',
    5: 'bus',
    7: 'truck'
}
VEHICLE_CLASSES = ['car', 'truck', 'bus', 'motorcycle', 'bicycle']
TWO_WHEEL_CLASSES = ['motorcycle', 'bicycle']
FOUR_WHEEL_CLASSES = ['car', 'truck', 'bus']
camera_keys = {}

BASE_DIR = '/kaggle/input/phat-trien-iot-nang-cao/pytorch/default/2/'
LP_DETECTOR_PATH = BASE_DIR + 'LP_detector.pt'
LP_OCR_PATH = BASE_DIR + 'LP_ocr.pt'
TL_MODEL_PATH = BASE_DIR + 'mhiot-dentinhieu-best-new.pt'

print(f"📡 Backend: {BACKEND_URL}")

In [ ]:
# Cell 2: Install
!pip uninstall -y numpy pillow ultralytics > /dev/null 2>&1
!pip install "numpy<2.0.0" "pillow>=10.3.0" scipy ultralytics opencv-python-headless requests websocket-client supervision --upgrade --quiet
!wget -nc -q https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11m.pt

import numpy, PIL, supervision
print(f'✅ Numpy {numpy.__version__}, PIL {PIL.__version__}, Supervision {supervision.__version__}')

In [ ]:
# Cell 3: Load Models
import warnings
warnings.filterwarnings('ignore')
import os, torch, re
from ultralytics import YOLO

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'🚀 Device: {device}')

vehicle_model = None
traffic_light_model = None
lp_detector = None
lp_ocr = None

def load_yolov5_model(path, name):
    if not os.path.exists(path):
        print(f'❌ {name}: Not found')
        return None
    try:
        model = torch.hub.load('ultralytics/yolov5', 'custom', path=path, force_reload=False)
        model = model.to(device)
        model.conf = 0.4
        model.iou = 0.45
        print(f'✅ {name}: YOLOv5')
        return model
    except Exception as e:
        print(f'❌ {name}: {e}')
        return None

def load_ultralytics_model(path, name):
    if not os.path.exists(path):
        print(f'❌ {name}: Not found')
        return None
    try:
        model = YOLO(path).to(device)
        print(f'✅ {name}: Ultralytics')
        return model
    except Exception as e:
        print(f'❌ {name}: {e}')
        return None

vehicle_model = YOLO('yolo11m.pt').to(device)
print('✅ Vehicle: yolo11m.pt')

traffic_light_model = load_ultralytics_model(TL_MODEL_PATH, 'Traffic Light')
lp_detector = load_yolov5_model(LP_DETECTOR_PATH, 'LP Detector')
lp_ocr = load_yolov5_model(LP_OCR_PATH, 'LP OCR')

print('\n=== MODELS ===')
print(f'Vehicle: {"✅" if vehicle_model else "❌"}')
print(f'TL: {"✅" if traffic_light_model else "❌"}')
print(f'LP: {"✅" if lp_detector else "❌"}')
print(f'OCR: {"✅" if lp_ocr else "❌"}')

In [ ]:
# Cell 4: Functions - Track ID Counting
import cv2, numpy as np, time, requests, threading, queue, json
from datetime import datetime
from collections import deque, defaultdict, Counter
import supervision as sv

camera_trackers = {}
camera_positions = {}
camera_configs = {}
current_traffic_light = {}

violation_history = {}
track_plate_votes = defaultdict(lambda: defaultdict(Counter))

# Track ID counting - unique track_ids seen per camera per type
# {camera_id: {'car': set(), 'truck': set(), ...}}
seen_tracks = defaultdict(lambda: defaultdict(set))

# Stats for logging
camera_stats = defaultdict(lambda: {
    'last_log_time': 0,
    'vehicle_counts': {'car': 0, 'truck': 0, 'bus': 0, 'motorcycle': 0, 'bicycle': 0},
    'plates_detected': set(),
    'traffic_light': 'UNKNOWN',
    'violations': [],
    'fps': 0
})

PLATE_REGEX_2WHEEL = re.compile(r'^\d{2}[A-Z]{2}\d{5}$')
PLATE_REGEX_4WHEEL = re.compile(r'^\d{2}[A-Z]\d{4,5}$')

def get_vehicle_class(coco_class_id):
    return COCO_CLASS_MAP.get(coco_class_id, 'car')

def get_track_line_y(camera_id):
    config = camera_configs.get(camera_id, {})
    return config.get('camera_track_line_y', DEFAULT_TRACK_LINE_Y)

def get_track_counts(camera_id):
    """Get unique track counts by vehicle type"""
    counts = seen_tracks[camera_id]
    return {
        'car': len(counts['car']),
        'truck': len(counts['truck']),
        'bus': len(counts['bus']),
        'motorcycle': len(counts['motorcycle']),
        'bicycle': len(counts['bicycle'])
    }

def add_track(camera_id, track_id, vehicle_class):
    """Add track_id to seen tracks"""
    seen_tracks[camera_id][vehicle_class].add(track_id)

def validate_plate_format(plate, vehicle_class):
    if not plate or len(plate) < 7:
        return False
    plate = plate.upper().strip()
    if vehicle_class.lower() in TWO_WHEEL_CLASSES:
        return bool(PLATE_REGEX_2WHEEL.match(plate))
    elif vehicle_class.lower() in FOUR_WHEEL_CLASSES:
        return bool(PLATE_REGEX_4WHEEL.match(plate))
    return bool(PLATE_REGEX_2WHEEL.match(plate) or PLATE_REGEX_4WHEEL.match(plate))

def log(msg):
    print(f'[{datetime.now().strftime("%H:%M:%S")}] {msg}')

def get_best_plate_for_track(camera_id, track_id, vehicle_class):
    votes = track_plate_votes[camera_id].get(track_id)
    if votes and len(votes) > 0:
        for plate, count in votes.most_common():
            if count >= 2 and validate_plate_format(plate, vehicle_class):
                return plate
    return None

def vote_plate_for_track(camera_id, track_id, plate, vehicle_class):
    if plate and plate != 'unknown' and validate_plate_format(plate, vehicle_class):
        track_plate_votes[camera_id][track_id][plate] += 1

def cleanup_old_tracks(camera_id, active_track_ids):
    to_delete = [tid for tid in track_plate_votes[camera_id] if tid not in active_track_ids]
    for tid in to_delete:
        del track_plate_votes[camera_id][tid]

def can_record_violation(plate, violation_type):
    key = f'{plate}:{violation_type}'
    now = time.time()
    if key in violation_history:
        if now - violation_history[key] < VIOLATION_COOLDOWN:
            return False
    violation_history[key] = now
    to_delete = [k for k, v in violation_history.items() if now - v > 60]
    for k in to_delete:
        del violation_history[k]
    return True

def print_summary(camera_id):
    stats = camera_stats[camera_id]
    now = time.time()
    if now - stats['last_log_time'] < LOG_INTERVAL:
        return
    stats['last_log_time'] = now
    
    # Get total unique tracks (cumulative)
    track_counts = get_track_counts(camera_id)
    total_tracks = sum(track_counts.values())
    
    vc = stats['vehicle_counts']  # Current frame counts
    current_total = sum(vc.values())
    vehicles = f"🚗{vc['car']} 🚚{vc['truck']} 🚌{vc['bus']} 🏍️{vc['motorcycle']}"
    
    plates = list(stats['plates_detected'])[-5:]
    plates_str = ', '.join(plates) if plates else 'none'
    tl = stats['traffic_light']
    tl_icon = '🔴' if tl == 'RED' else '🟢' if tl == 'GREEN' else '🟡' if tl == 'YELLOW' else '⚪'
    viols = stats['violations'][-3:]
    viols_str = ', '.join([f"{v['type']}:{v['plate']}" for v in viols]) if viols else 'none'
    
    log(f"\n{'='*50}")
    log(f"📷 Camera [{camera_id[-4:]}] | {stats['fps']:.1f} FPS | Line: {get_track_line_y(camera_id)}%")
    log(f"🚦 Traffic Light: {tl_icon} {tl}")
    log(f"🚗 Current ({current_total}): {vehicles}")
    log(f"📊 TOTAL TRACKS: {total_tracks} | 🚗{track_counts['car']} 🚚{track_counts['truck']} 🚌{track_counts['bus']} 🏍️{track_counts['motorcycle']}")
    log(f"📋 Plates: {plates_str}")
    log(f"⚠️ Violations: {viols_str}")
    log(f"{'='*50}\n")
    
    # Reset current counts (but NOT total tracks)
    stats['vehicle_counts'] = {'car': 0, 'truck': 0, 'bus': 0, 'motorcycle': 0, 'bicycle': 0}
    stats['plates_detected'] = set()

def read_plate_ocr(plate_crop):
    if lp_ocr is None:
        return 'unknown'
    try:
        h, w = plate_crop.shape[:2]
        if h < 10 or w < 20:
            return 'unknown'
        lp_ocr.conf = 0.25
        results = lp_ocr(plate_crop)
        df = results.pandas().xyxy[0]
        if len(df) < 4:
            return 'unknown'
        df['y_center'] = (df['ymin'] + df['ymax']) / 2
        y_threshold = h / 2
        line1 = df[df['y_center'] < y_threshold].sort_values('xmin')
        line2 = df[df['y_center'] >= y_threshold].sort_values('xmin')
        chars = line1['name'].astype(str).tolist() + line2['name'].astype(str).tolist()
        result = ''.join(chars).upper().replace(' ', '').replace('-', '')
        return result if len(result) >= 7 else 'unknown'
    except:
        return 'unknown'

def detect_license_plate(frame, x1, y1, x2, y2):
    if lp_detector is None:
        return None
    try:
        crop = frame[y1:y2, x1:x2]
        lp_detector.conf = 0.4
        results = lp_detector(crop)
        df = results.pandas().xyxy[0]
        if len(df) == 0:
            return None
        best = df.iloc[df['confidence'].idxmax()]
        px1, py1, px2, py2 = int(best['xmin']), int(best['ymin']), int(best['xmax']), int(best['ymax'])
        plate_crop = crop[py1:py2, px1:px2]
        text = read_plate_ocr(plate_crop)
        if text != 'unknown':
            return {'text': text, 'confidence': float(best['confidence'])}
        return None
    except:
        return None

def check_red_light_violation(track_id, prev_y, curr_y, camera_id, img_height):
    if current_traffic_light.get(camera_id, 'UNKNOWN') != 'RED':
        return False
    track_line_y = get_track_line_y(camera_id)
    line_y = (track_line_y / 100) * img_height
    if prev_y is not None and prev_y > line_y >= curr_y:
        return True
    return False

def check_lane_violation(vehicle_class, center_x, camera_id, img_width):
    config = camera_configs.get(camera_id)
    if not config:
        return False
    lane_points = config.get('camera_lane_track_point', [])
    lane_vehicles = config.get('camera_lane_vehicles', [])
    if not lane_points or not lane_vehicles:
        return False
    x_pct = (center_x / img_width) * 100
    boundaries = [0] + sorted(lane_points) + [100]
    for i in range(len(boundaries) - 1):
        if boundaries[i] <= x_pct < boundaries[i + 1]:
            if i < len(lane_vehicles):
                allowed = lane_vehicles[i]
                if isinstance(allowed, str):
                    if allowed.upper() in ['*', 'ANY', 'ALL']:
                        return False
                    if vehicle_class.lower() != allowed.lower():
                        return True
                    return False
                elif isinstance(allowed, list):
                    allowed_upper = [str(v).upper() for v in allowed]
                    if '*' in allowed_upper or 'ANY' in allowed_upper or 'ALL' in allowed_upper:
                        return False
                    allowed_lower = [str(v).lower() for v in allowed]
                    if vehicle_class.lower() not in allowed_lower:
                        return True
                    return False
            break
    return False

In [ ]:
# Cell 5: Main Detection - Track ID Counting

def detect_frame(frame, camera_id):
    global current_traffic_light
    stats = camera_stats[camera_id]
    
    h, w = frame.shape[:2]
    
    # Get current track counts (cumulative)
    track_counts = get_track_counts(camera_id)
    
    result = {
        'camera_id': camera_id,
        'created_at': int(time.time() * 1000),
        'image_dimensions': {'width': w, 'height': h},
        'detections': [],
        'vehicle_count': 0,
        'tracks': [],
        'track_counts': track_counts,  # Send cumulative track counts
        'license_plates': [],
        'violations': []
    }
    
    if camera_id not in camera_trackers:
        camera_trackers[camera_id] = sv.ByteTrack()
        camera_positions[camera_id] = {}
    
    tracker = camera_trackers[camera_id]
    positions = camera_positions[camera_id]
    
    # Traffic Light
    if traffic_light_model:
        try:
            tl_results = traffic_light_model(frame, conf=0.4, verbose=False)
            status = 'UNKNOWN'
            tl_dets = []
            for b in tl_results[0].boxes:
                cls_name = tl_results[0].names[int(b.cls[0])].lower()
                bx1, by1, bx2, by2 = map(float, b.xyxy[0])
                tl_dets.append({'class': cls_name, 'confidence': float(b.conf[0]), 'bbox': {'x1': bx1/w, 'y1': by1/h, 'x2': bx2/w, 'y2': by2/h}})
                if 'red' in cls_name: status = 'RED'
                elif 'green' in cls_name and status != 'RED': status = 'GREEN'
                elif 'yellow' in cls_name and status == 'UNKNOWN': status = 'YELLOW'
            result['traffic_light'] = {'status': status, 'detections': tl_dets}
            current_traffic_light[camera_id] = status
            stats['traffic_light'] = status
        except: pass
    
    # Vehicle
    if vehicle_model:
        try:
            v_results = vehicle_model(frame, classes=[2,3,5,7], conf=CONFIDENCE, verbose=False)[0]
            sv_dets = sv.Detections.from_ultralytics(v_results)
            tracked = tracker.update_with_detections(sv_dets)
            
            detections = []
            tracks_dict = {}
            active_track_ids = set()
            
            for i in range(len(tracked)):
                x1, y1, x2, y2 = map(int, tracked.xyxy[i])
                cx, cy = (x1+x2)//2, (y1+y2)//2
                
                coco_cls_id = int(tracked.class_id[i]) if tracked.class_id is not None else 2
                cls = get_vehicle_class(coco_cls_id)
                
                conf = float(tracked.confidence[i]) if tracked.confidence is not None else 0.5
                track_id = int(tracked.tracker_id[i]) if tracked.tracker_id is not None else -1
                
                active_track_ids.add(track_id)
                
                # Add to seen tracks (cumulative counting)
                add_track(camera_id, track_id, cls)
                
                # Count current frame
                if cls in stats['vehicle_counts']:
                    stats['vehicle_counts'][cls] += 1
                
                det = {
                    'class': cls,
                    'confidence': conf,
                    'bbox': {'x1': x1/w, 'y1': y1/h, 'x2': x2/w, 'y2': y2/h},
                    'center': {'x': cx, 'y': cy},
                    'track_id': track_id
                }
                tracks_dict[track_id] = {'pos': (cx, cy), 'class': cls}
                
                # LP
                area = (x2-x1)*(y2-y1)
                if area > (w*h)*0.005:
                    lp_info = detect_license_plate(frame, x1, y1, x2, y2)
                    if lp_info:
                        vote_plate_for_track(camera_id, track_id, lp_info['text'], cls)
                
                best_plate = get_best_plate_for_track(camera_id, track_id, cls)
                if best_plate:
                    det['license_plate'] = best_plate
                    stats['plates_detected'].add(best_plate)
                    if best_plate not in [p['plate'] for p in result['license_plates']]:
                        result['license_plates'].append({'plate': best_plate, 'vehicle_id': track_id, 'confidence': 0.9})
                
                # RED LIGHT
                prev_y = positions.get(track_id)
                if check_red_light_violation(track_id, prev_y, cy, camera_id, h):
                    if best_plate and can_record_violation(best_plate, 'RED_LIGHT'):
                        result['violations'].append({'type': 'RED_LIGHT', 'license_plate': best_plate, 'bbox': det['bbox'], 'detection_id': track_id})
                        stats['violations'].append({'type': 'RED_LIGHT', 'plate': best_plate})
                        log(f'🚨 VIOLATION: RED_LIGHT - {best_plate}')
                
                # LANE
                if check_lane_violation(cls, cx, camera_id, w):
                    if best_plate and can_record_violation(best_plate, 'LANE'):
                        result['violations'].append({'type': 'LANE', 'license_plate': best_plate, 'bbox': det['bbox'], 'detection_id': track_id})
                        stats['violations'].append({'type': 'LANE', 'plate': best_plate})
                        log(f'🚨 VIOLATION: LANE - {best_plate}')
                
                positions[track_id] = cy
                detections.append(det)
            
            cleanup_old_tracks(camera_id, active_track_ids)
            
            result['detections'] = detections
            result['vehicle_count'] = len(detections)
            result['track_counts'] = get_track_counts(camera_id)  # Update with latest
            result['tracks'] = [{'id': tid, 'class': info['class'], 'positions': [{'x': info['pos'][0], 'y': info['pos'][1], 'time': result['created_at']}]} for tid, info in tracks_dict.items()]
        except Exception as e:
            log(f'Vehicle err: {e}')
    
    print_summary(camera_id)
    return result

In [ ]:
# Cell 6: WebSocket
import websocket, ssl

class KaggleWebSocket:
    def __init__(self, ws_url, api_key):
        self.url = f'{ws_url}?apiKey={api_key}'
        self.ws = None
        self.connected = False
        self.send_queue = queue.Queue(maxsize=RESULT_QUEUE_SIZE)
        self._stop = False
    
    def connect(self):
        try:
            log(f'🔌 Connecting...')
            self.ws = websocket.create_connection(self.url, timeout=10, sslopt={'cert_reqs': ssl.CERT_NONE})
            self.connected = True
            log('✅ WS connected!')
            threading.Thread(target=self._sender_loop, daemon=True).start()
            return True
        except Exception as e: log(f'❌ WS: {e}'); return False
    
    def _sender_loop(self):
        while not self._stop:
            try:
                msg = self.send_queue.get(timeout=1)
                if self.ws and self.connected: self.ws.send(json.dumps(msg))
            except queue.Empty: continue
            except Exception as e: log(f'WS err: {e}'); self.connected = False
    
    def send_async(self, data):
        try: self.send_queue.put_nowait(data)
        except: pass

def fetch_cameras():
    global camera_configs, camera_keys
    try:
        resp = requests.get(f'{BACKEND_HTTP_URL}/api/camera/all', timeout=10)
        if resp.status_code == 200:
            data = resp.json()
            cams = data.get('metadata', data) if isinstance(data, dict) else data
            for cam in cams:
                if isinstance(cam, dict) and cam.get('_id'):
                    cam_id = cam['_id']
                    camera_keys[cam_id] = cam.get('camera_api_key', '')
                    camera_configs[cam_id] = {
                        'camera_track_line_y': cam.get('camera_track_line_y', 50),
                        'camera_lane_track_point': cam.get('camera_lane_track_point', []),
                        'camera_lane_vehicles': cam.get('camera_lane_vehicles', [])
                    }
                    log(f'📹 Camera {cam_id[-4:]}: track_line_y={camera_configs[cam_id]["camera_track_line_y"]}%')
            return list(camera_keys.keys())
    except Exception as e: log(f'Fetch err: {e}')
    return []

def test_stream(url, timeout=5):
    try:
        cap = cv2.VideoCapture(url)
        cap.set(cv2.CAP_PROP_OPEN_TIMEOUT_MSEC, timeout*1000)
        if cap.isOpened(): ret, _ = cap.read(); cap.release(); return ret
        return False
    except: return False

In [ ]:
# Cell 7: Pipeline

class CameraPipeline:
    def __init__(self, camera_id, api_key, kaggle_ws):
        self.camera_id = camera_id
        self.flv_url = f"{NMS_URL}/live/{camera_id}.flv"
        self.frame_queue = queue.Queue(maxsize=FRAME_QUEUE_SIZE)
        self.kaggle_ws = kaggle_ws
        self._stop = False
        self.fps_times = deque(maxlen=30)
        self.stream_ready = False
    
    def start(self):
        log(f'🎥 [{self.camera_id[-4:]}] Testing...')
        if not test_stream(self.flv_url, timeout=10): log(f'⚠️ [{self.camera_id[-4:]}] No stream')
        threading.Thread(target=self._reader_loop, daemon=True).start()
        threading.Thread(target=self._detector_loop, daemon=True).start()
        log(f'🚀 [{self.camera_id[-4:]}] Started')
        return True
    
    def _reader_loop(self):
        retry = 2
        while not self._stop:
            try:
                cap = cv2.VideoCapture(self.flv_url)
                cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
                if not cap.isOpened(): time.sleep(retry); retry = min(retry*2, 30); continue
                self.stream_ready = True; retry = 2
                while cap.isOpened() and not self._stop:
                    ret, frame = cap.read()
                    if not ret: break
                    try: self.frame_queue.put_nowait(frame)
                    except: 
                        try: self.frame_queue.get_nowait(); self.frame_queue.put_nowait(frame)
                        except: pass
                cap.release(); self.stream_ready = False
            except Exception as e: log(f'Reader: {e}'); time.sleep(retry)
    
    def _detector_loop(self):
        while not self._stop:
            try:
                frame = self.frame_queue.get(timeout=2)
                result = detect_frame(frame, self.camera_id)
                self.kaggle_ws.send_async(result)
                
                now = time.time()
                self.fps_times.append(now)
                if len(self.fps_times) > 1:
                    camera_stats[self.camera_id]['fps'] = len(self.fps_times) / (self.fps_times[-1] - self.fps_times[0])
            except queue.Empty: continue
            except Exception as e: log(f'Det: {e}')

In [ ]:
# Cell 8: MAIN
log('🔍 Fetching cameras...')
cameras = fetch_cameras()

if not cameras:
    log('❌ No cameras!')
else:
    log(f'✅ Found {len(cameras)} cameras')
    pipelines = []
    for cam_id in cameras:
        ws = KaggleWebSocket(BACKEND_WS_URL, camera_keys.get(cam_id, ''))
        if not ws.connect(): continue
        p = CameraPipeline(cam_id, camera_keys.get(cam_id, ''), ws)
        if p.start(): pipelines.append(p)
        time.sleep(0.5)
    
    log(f'🚀 {len(pipelines)} PIPELINES RUNNING!')
    try:
        while True:
            time.sleep(60)
    except KeyboardInterrupt:
        log('Stop')